In [1]:
# get rid of warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from Evaluation_Metriken import JaccardEval
from Evaluation_Metriken import SorensenEval
from Evaluation_Metriken import HammingDist
import pickle 


In [2]:
# Load data for rank model
data = pd.read_csv(r"..\preprocessedData\hdgesamt_nodup.csv", sep=",")
# Load data for sequence model
data_seq = pd.read_csv(r"..\preprocessedData\hdgesamt_seq.csv", sep=",")
# Load data Verweil
data_verweil = pd.read_csv(r"..\preprocessedData\hdgesamt_verweil.csv", sep=",")
# Sort data (before merge)
data = data.sort_values(by=["Fall_ID","rank_min","OPSlabel"])

In [3]:
# Test Entry
X_demo = pd.Series(data=
    {'ICD': 'I48.10',
     'Geschlecht' : 'm', 
     'OPS_Altergruppe' : 7})
t = 10

In [4]:
# retrieve all the cp in the data set
allstatKBP = data.groupby(["Fall_ID","ICD",'Geschlecht',"OPS_Altergruppe"], as_index = False)['OPSlabel'].agg(pd.Series.tolist) #as_index is the key ...
# retrieve all the cp related to the test ticket
statKBP=allstatKBP[(allstatKBP['ICD'] == X_demo[0])&(allstatKBP['Geschlecht'] == X_demo[1])&(allstatKBP['OPS_Altergruppe'] == X_demo[2])]
# sample cp
sample_maxfreq = statKBP['OPSlabel'].describe().top

In [5]:
# find the age with max freq for ticket
def findAgeMaxFreq(X_demo):
    maxfreq_age=allstatKBP[(allstatKBP['ICD'] == X_demo[0])&(allstatKBP['Geschlecht'] == X_demo[1])].groupby(['OPS_Altergruppe'], as_index=False).count()
    maxfreq_age = maxfreq_age[['OPS_Altergruppe','Fall_ID']]
    maxfreq_age.rename(columns={"Fall_ID": "Fälle"}, inplace=True)
    return maxfreq_age
findAgeMaxFreq(X_demo)

,OPS_Altergruppe,Fälle
0,4,2
1,5,13
2,6,56
3,7,54
4,8,5


In [6]:
# find the most frequent diagnosis
def findPopularDiseaseGender(minNoCase, gender, g1, g2):
    statKBP_gender=allstatKBP[allstatKBP['Geschlecht']==gender].groupby(['ICD','OPS_Altergruppe'],as_index=False).count()
    group1 = statKBP_gender[(statKBP_gender['OPS_Altergruppe'] == g1)][['OPS_Altergruppe','ICD','Geschlecht']]
    group2 = statKBP_gender[(statKBP_gender['OPS_Altergruppe'] == g2)][['OPS_Altergruppe','ICD','Geschlecht']]
    mergedGroup = pd.merge(group1, group2, on=['ICD'], how='inner', suffixes=('_g1', '_g2'))
    mergedGroup.rename(columns={"Geschlecht_g1": "Fälle_g1",
                                "Geschlecht_g2": "Fälle_g2"}, inplace=True)
    mergedGroup_short = mergedGroup[(mergedGroup['Fälle_g1']>minNoCase)&(mergedGroup['Fälle_g2']>minNoCase)]
    mergedGroup_short = mergedGroup_short[['ICD','OPS_Altergruppe_g1','Fälle_g1','OPS_Altergruppe_g2','Fälle_g2']]
    return mergedGroup_short
findPopularDiseaseGender(10, 'w', 4, 7)

,ICD,OPS_Altergruppe_g1,Fälle_g1,OPS_Altergruppe_g2,Fälle_g2
12,C73,4,13,7,26
29,E04.2,4,19,7,45
55,H04.5,4,20,7,38
72,I47.1,4,18,7,13


In [7]:
# Find the most common diseases for 2 groups
def findPopularDiseaseAge(g1,g2,top_x):
    allstatKBPgrouped= allstatKBP[(allstatKBP['OPS_Altergruppe']==g1)|(allstatKBP['OPS_Altergruppe']==g2)]
    freqDiseaseGrouped = allstatKBPgrouped.groupby(['ICD'], as_index = False).count().sort_values(by='Fall_ID', ascending=False).head(top_x)
    freqDiseaseGrouped = freqDiseaseGrouped[['ICD','Fall_ID']]
    freqDiseaseGrouped.rename(columns={"Fall_ID": "Fälle"}, inplace=True)
    return freqDiseaseGrouped
findPopularDiseaseAge(6,7,20)

,ICD,Fälle
202,C61,344
795,I25.13,289
1392,M54.4,274
431,E04.2,222
832,I48.10,200
1289,M17.1,192
780,I20.0,176
1519,N40,158
1284,M16.1,141
669,H25.8,133


1a. Run the first submodel to predict OPS

In [8]:
# Transform X_demo to list
X_demo = [X_demo]

# Define y and X
y = data['OPSlabel']
X = data[['ICD','Geschlecht','OPS_Altergruppe']]

# Encode declaration, fit and transform X first
Encoder = OneHotEncoder(handle_unknown="ignore")
X_fit = Encoder.fit_transform(X)

# Load trained model to predict OPS
filename = 'OPS_Modell.pkl'
clf = pickle.load(open(filename, 'rb'))

# Test with demo ticket
X_demo_transform = Encoder.transform(X_demo)
probs = clf.predict_proba(X_demo_transform)

# create list of all labels sorted after probabilities descendingly
labelidx = np.argsort(-probs, axis=1)
prob = []
[probs] = probs
[labelidx] = labelidx
prob = probs.take(labelidx)
prob = [value*100 for value in prob]

# transform numberic value to label (y)
le = LabelEncoder()
y_fit = le.fit_transform(y)
all_label = le.inverse_transform(labelidx)

# result as a dataframe
fullresult = pd.DataFrame({"OPS": all_label,
    "Probability": prob})

# Threshold implementation 
best_result=fullresult[fullresult.Probability > t]

1b. Run the second submodel to predict rank_min for each predicted OPS

In [9]:
# Load model which predict rank
filename1 = 'Rank_model.pkl'
clf1 = pickle.load(open(filename1, 'rb'))

# Define y and X
y1 = data['rank_min']
X1 = data[['ICD','Geschlecht','OPS_Altergruppe','OPSlabel']]

# Encode declaration, fit and transform X first
Encoder1 = OneHotEncoder(handle_unknown="ignore")
X_fit1 = Encoder1.fit_transform(X1)

# fit y1 to do inverse transform later on
le1 = LabelEncoder()
y_fit1 = le1.fit_transform(y1)

# Do the same thing as for the first submodel, but for all OPS predicted
predicted_rank = []

for i in range(len(best_result)):
# test entry edited
    X_demo1 = pd.Series(data={'ICD': X_demo[0][0], 'Geschlecht' : X_demo[0][1], 'OPS_Altergruppe' : X_demo[0][2], 
        'OPSlabel': best_result.OPS[i]})
    X_demo1 = [X_demo1]
    X_demo_transform1 = Encoder1.transform(X_demo1)
    probs1 = clf1.predict_proba(X_demo_transform1)
    best_n1 = np.argsort(probs1, axis=1)[:,-1:]
    [probs1] = probs1
    [best_n1] = best_n1
    prob1 = probs[best_n1[0]] # but this time only the best label was consider
    inversed_label = le1.inverse_transform(best_n1)[0]
    predicted_rank.append(inversed_label)

# Convert the predicted rank list to series to concatenate
predicted_rank_series = pd.Series(predicted_rank, name='Rank')

# Concatenate to create the first clinical pathway
clinical_pathway1= pd.concat([best_result,predicted_rank_series],axis=1).sort_values(by=['Rank','OPS'], ascending=[True,True])
clinical_pathway1=clinical_pathway1.reset_index().drop('index', axis=1)

# Calculate LOS (Length of stay) using data_verweil
vwd = int(data_verweil.loc[data_verweil.ICD == X_demo[0][0]]['Verweildauer'])

# Create the column  "Day of Treatment"
clinical_pathway1['Day Of Treatment'] = ((clinical_pathway1['Rank']*vwd)/100).astype(int).astype(str) + " of " + str(vwd)

2. Run the second model

In [10]:
# Define y and X
y2 = data_seq['OPSlabel']
X2 = data_seq[['ICD','Geschlecht','OPS_Altergruppe','OPSprev']]

# Encode declaration, fit and transform X first
Encoder2 = OneHotEncoder(handle_unknown="ignore")
X_fit2 = Encoder2.fit_transform(X2)

# LabelEncoder for y later on
le2 = LabelEncoder()
y_fit2 = le2.fit_transform(y2)
# load model 2
filename2 = 'Seq_modell.pkl'
clf2 = pickle.load(open(filename2, 'rb'))

# Function to generate the next label 
def nextLabel(x,i):    
    x = [x]
    X_transform = Encoder2.transform(x)
    probs = clf2.predict_proba(X_transform)
    labelidx = np.argsort(-probs, axis=1)
    [probs] = probs
    [labelidx] = labelidx
    all_label = le2.inverse_transform(labelidx)
    return [all_label[i],probs[labelidx[i]]]

# Test Entry edited
X_demo2 = pd.Series(data={'ICD': X_demo[0][0], 'Geschlecht' : X_demo[0][1], 'OPS_Altergruppe' : X_demo[0][2], 'OPSprev' : 'S'})

# initialise the first row of clinical pathway 2
start = ['S', 1.0]
clinical_pathway2 = pd.Series(data={'OPS': start[0] ,'Probability': start[1]})

# call the function above, first entry
a = nextLabel(X_demo2,0)
clinical_pathway2 = pd.concat([clinical_pathway2,pd.Series(data={'OPS': a[0] ,'Probability': a[1]})], axis=1)
[icd,sex,age] = [X_demo2['ICD'],X_demo2['Geschlecht'],X_demo2['OPS_Altergruppe']]

# keep generate the next label until "E" comes up, then concatenate the generated with the clinical pathway
while (nextLabel(X_demo2,0)[0] != 'E'):
    X_demo2 = pd.Series(data={'ICD': icd, 'Geschlecht' : sex, 'OPS_Altergruppe' : age, 'OPSprev' : nextLabel(X_demo2,0)[0]})
    if (nextLabel(X_demo2,0)[0] not in clinical_pathway2.iloc[0].values):
        b =  pd.Series(data={'OPS': nextLabel(X_demo2,0)[0] ,'Probability': nextLabel(X_demo2,0)[1]})
        clinical_pathway2 = pd.concat([clinical_pathway2,b], axis=1)
    else:
        b =  pd.Series(data={'OPS': 'E' ,'Probability': 1.0})
        clinical_pathway2 = pd.concat([clinical_pathway2,b], axis=1)
        break
# clean the dataframe
clinical_pathway2 = clinical_pathway2.T.reset_index().drop('index', axis=1)
clinical_pathway2 = clinical_pathway2[1:-1]

3. Metrics

In [11]:
# Metric 1: matching_accuracy_sorensen
def matching_accuracy_sorensen(cp):
    try:
        genKBP = cp['OPS'].array
        genKBP = set(genKBP)
        genKBPlen = len(genKBP)
        allstatKBP = data.groupby(["Fall_ID","ICD",'Geschlecht',"OPS_Altergruppe"], as_index = False)['OPSlabel'].agg(pd.Series.tolist) #as_index is the key ...
        statKBP=allstatKBP[(allstatKBP['ICD'] == X_demo[0][0])&(allstatKBP['Geschlecht'] == X_demo[0][1])&(allstatKBP['OPS_Altergruppe'] == X_demo[0][2])]
        statKBP_maxfreq=set(statKBP['OPSlabel'].describe().top)
        statKBP_maxfreqlen = len(statKBP_maxfreq)
        inter = len(statKBP_maxfreq.intersection(genKBP))
        Sorensen = round(SorensenEval(inter,genKBPlen,statKBP_maxfreqlen),4)
    except:
        Sorensen = "no precedent"
    return Sorensen

In [12]:
# Metric 2: matching_accuracy_jaccard
def matching_accuracy_jaccard(cp):
    try:
        genKBP = cp['OPS'].array
        genKBP = set(genKBP)
        genKBPlen = len(genKBP)
        allstatKBP = data.groupby(["Fall_ID","ICD",'Geschlecht',"OPS_Altergruppe"], as_index = False)['OPSlabel'].agg(pd.Series.tolist) #as_index is the key ...
        statKBP=allstatKBP[(allstatKBP['ICD'] == X_demo[0][0])&(allstatKBP['Geschlecht'] == X_demo[0][1])&(allstatKBP['OPS_Altergruppe'] == X_demo[0][2])]
        statKBP_maxfreq=set(statKBP['OPSlabel'].describe().top)
        statKBP_maxfreqlen = len(statKBP_maxfreq)
        inter = len(statKBP_maxfreq.intersection(genKBP))
        Jaccard = round(JaccardEval(inter,genKBPlen,statKBP_maxfreqlen),4)
    except:
        Jaccard = "no precedent"
    return Jaccard

In [13]:
# Metric 3: Hamming distance, position 
def hammingDist(cp):
    try:
        genKBP = cp['OPS'].array
        genKBPlen = len(genKBP)
        allstatKBP = data.groupby(["Fall_ID","ICD",'Geschlecht',"OPS_Altergruppe"], as_index = False)['OPSlabel'].agg(pd.Series.tolist) #as_index is the key ...
        statKBP=allstatKBP[(allstatKBP['ICD'] == X_demo[0][0])&(allstatKBP['Geschlecht'] == X_demo[0][1])&(allstatKBP['OPS_Altergruppe'] == X_demo[0][2])]
        statKBP['KBPlen'] = statKBP['OPSlabel'].apply(lambda x: len(x))
        statKBPsameLen = statKBP[statKBP['KBPlen'] == genKBPlen]
        statKBPsameLen_maxfreq = statKBPsameLen['OPSlabel'].describe().top
        hammingDist = HammingDist(genKBP,statKBPsameLen_maxfreq)
    except:
        hammingDist = "no precedent with same length"
    return hammingDist

In [14]:
# Metric 4: How many cases did follow this clinical pathways
def no_sim_cp(cp):
    try:
        genKBP = cp['OPS'].tolist()
        genKBPlen = len(genKBP)
        allstatKBP = data.groupby(["Fall_ID","ICD",'Geschlecht',"OPS_Altergruppe"], as_index = False)['OPSlabel'].agg(pd.Series.tolist) #as_index is the key ...
        statKBP=allstatKBP[(allstatKBP['ICD'] == X_demo[0][0])&(allstatKBP['Geschlecht'] == X_demo[0][1])&(allstatKBP['OPS_Altergruppe'] == X_demo[0][2])]
        statKBP['KBPlen'] = statKBP['OPSlabel'].apply(lambda x: len(x))
        statKBPsameLen = statKBP[statKBP['KBPlen'] == genKBPlen]
        simCp = statKBPsameLen[statKBPsameLen.OPSlabel.apply(lambda x: x==genKBP)]
        simCplen = len(simCp)
    except:
        simCplen = "no precedent"
    return simCplen

In [15]:
# Accuracy table
model_name = ['1','2']
hammingDist_arr = [ hammingDist(clinical_pathway1), hammingDist(clinical_pathway2)]
no_sim_arr = [ no_sim_cp(clinical_pathway1), no_sim_cp(clinical_pathway2)]
matching_accuracy_jaccard_arr = [matching_accuracy_jaccard(clinical_pathway1), matching_accuracy_jaccard(clinical_pathway2)]
matching_accuracy_sorensen_arr = [matching_accuracy_sorensen(clinical_pathway1), matching_accuracy_sorensen(clinical_pathway2)]
len_arr = [len(clinical_pathway1),len(clinical_pathway2)]
accuracy_table = pd.DataFrame(
    {"Modell": model_name,
    "Jaccard": matching_accuracy_jaccard_arr,
    "Sorensen": matching_accuracy_sorensen_arr,
    "Hamming": hammingDist_arr,
    "Häufigkeit": no_sim_arr,
    "Länge der KBP": len_arr,
    "Anzahl der Fälle": [len(statKBP),len(statKBP)]
    })
styles = [dict(selector="caption",
                       props=[("text-align", "left"),
                              ("font-size", "100%"),
                              ("color", 'white')])]
accuracy_table=accuracy_table.style.set_caption('ICD = ' + X_demo[0][0] + ', Geschlecht = ' + X_demo[0][1] + ', Altergruppe = '+ str(X_demo[0][2]) + ':').set_table_styles(styles)

In [16]:
accuracy_table

,Modell,Jaccard,Sorensen,Hamming,Häufigkeit,Länge der KBP,Anzahl der Fälle
0,1,0.200000,0.333300,0,3,4,54
1,2,0.142900,0.250000,0,8,6,54


In [17]:
clinical_pathway1

,OPS,Probability,Rank,Day Of Treatment
0,3-052,19.352391,0,0 of 3
1,1-268,11.727553,34,1 of 3
2,1-274,10.718353,34,1 of 3
3,8-835,11.711454,34,1 of 3


In [18]:
clinical_pathway2

,OPS,Probability
1,3-052,0.481529
2,3-224,0.335371
3,1-265,0.495539
4,1-268,0.973233
5,1-274,0.963289
6,8-835,0.861474


In [19]:
sample_maxfreq

['3-052', '8-640']

In [20]:
statKBP

,Fall_ID,ICD,Geschlecht,OPS_Altergruppe,OPSlabel
1048,2783363,I48.10,m,7,"[3-052, 8-640]"
1154,2784075,I48.10,m,7,"[3-052, 8-640, 1-710]"
1242,2784525,I48.10,m,7,"[3-052, 3-224, 1-265, 1-268, 1-274, 8-835]"
1861,2790407,I48.10,m,7,"[3-052, 3-224, 1-265, 1-268, 1-274, 8-835]"
1990,2791595,I48.10,m,7,"[3-052, 3-224, 1-265, 1-268, 1-274, 8-835, 8-9..."
2015,2791751,I48.10,m,7,"[1-275, 3-052, 3-605, 8-837, 8-83b]"
2353,2794935,I48.10,m,7,"[3-052, 8-640]"
3101,2801349,I48.10,m,7,"[3-052, 1-265, 1-268, 1-274, 3-224, 8-835]"
3542,2805466,I48.10,m,7,"[8-930, 8-980, 3-052]"
3550,2805580,I48.10,m,7,"[3-052, 8-640]"
